In [1]:
import numpy as np # linear algebra
import os
from os.path import join
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

Using TensorFlow backend.


Load images.

In [2]:
def img_paths(image_dir):
    image_filenames = os.listdir(image_dir)
    return [join(image_dir, filename) for filename in image_filenames if not filename.endswith('.DS_Store')]

In [3]:
train_normal_dir = '../input/chest_xray/chest_xray/train/NORMAL'
train_normal_paths = img_paths(train_normal_dir)
train_pneumonia_dir = '../input/chest_xray/chest_xray/train/PNEUMONIA'
train_pneumonia_paths = img_paths(train_pneumonia_dir)
test_normal_dir = '../input/chest_xray/chest_xray/test/NORMAL'
test_normal_paths = img_paths(test_normal_dir)
test_pneumonia_dir = '../input/chest_xray/chest_xray/test/PNEUMONIA'
test_pneumonia_paths = img_paths(test_pneumonia_dir)
val_normal_dir = '../input/chest_xray/chest_xray/val/NORMAL'
val_normal_paths = img_paths(val_normal_dir)
val_pneumonia_dir = '../input/chest_xray/chest_xray/val/PNEUMONIA'
val_pneumonia_paths = img_paths(val_pneumonia_dir)

Prepare data.

In [14]:
image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width), grayscale=True) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    #return preprocess_input(img_array)
    return img_array

def make_data(normal_paths, pneumonia_paths):
    normal = read_and_prep_images(normal_paths)
    pneumonia = read_and_prep_images(pneumonia_paths)
    #print("normal's shape: ", normal.shape)
    #print("pneumonia's shape: ", pneumonia.shape)
    X = np.concatenate((normal, pneumonia))
    y = np.concatenate((np.zeros([normal.shape[0]], dtype=int), np.ones([pneumonia.shape[0]], dtype=int)))
    return X, y

In [15]:
X_train, y_train = make_data(train_normal_paths, train_pneumonia_paths)
print("X_train's shape: ", X_train.shape)
print("y_train's shape: ", y_train.shape)
X_test, y_test = make_data(test_normal_paths, test_pneumonia_paths)
print("X_test's shape: ", X_test.shape)
print("y_test's shape: ", y_test.shape)

X_train's shape:  (5216, 224, 224, 1)
y_train's shape:  (5216,)
X_test's shape:  (624, 224, 224, 1)
y_test's shape:  (624,)


Define hyperparameters.

In [16]:
learning_rate = 0.0001
batch_size = 128
epochs = 5
dropout = 0.25
seed = 1

Model = (Conv2D + Maxpool) -> Conv2D -> FC -> FC -> softmax.

In [17]:
model = Sequential()

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(16, (4, 4), strides=(2, 2), activation='relu', input_shape=(image_size, image_size, 1)))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(dropout, seed=seed))

model.add(ZeroPadding2D((2, 2)))
model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu'))
model.add(Dropout(dropout, seed=seed))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(dropout, seed=seed))
model.add(Dense(1, activation='softmax'))

#sgd = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='binary_crossentropy', optimizer=sgd)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)
score = model.evaluate(X_test, y_test, batch_size=batch_size)

Epoch 1/5
5216/5216 [==============================] - 40s 8ms/step - loss: 4.0987 - acc: 0.7429
Epoch 2/5
5216/5216 [==============================] - 39s 7ms/step - loss: 4.0987 - acc: 0.7429
Epoch 3/5
5216/5216 [==============================] - 39s 8ms/step - loss: 4.0987 - acc: 0.7429
Epoch 4/5
5216/5216 [==============================] - 40s 8ms/step - loss: 4.0987 - acc: 0.7429
Epoch 5/5
624/624 [==============================] - 2s 3ms/step


Result:

In [18]:
print(score)

[5.978394016241433, 0.625]
